In [ ]:
"""
ECGR 5105 - Intro to Machine Learning
Final Project
Projection of GDP Growth in Conjunction with Internet Accessibility
Phillip Harmon - pharmon@uncc.edu
""";


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import optim, nn
from collections import OrderedDict


In [12]:
#Training Loop Function
def training_loop(x_t, y_t, x_v, y_v, model, loss_function, optimizer, epochs):
    training_loss = []
    validation_loss = []
    
    for epoch in range(1, epochs + 1):
        
        loss_t = loss_function( model(x_t), y_t)
        loss_v = loss_function( model(x_v), y_v)
        
        optimizer.zero_grad()
        loss_t.backward()
        optimizer.step()
        
        training_loss.append(float(loss_t))
        validation_loss.append(float(loss_v))
        
        if epoch <= 3 or epoch % 50 == 0:
            print('Epoch {} | Training Loss = {} | Validation Loss = {}'.format(epoch, loss_t, loss_v))
    
    return training_loss, validation_loss


In [13]:
#helper for plotting visualization of training data
def training_visual(loss_t, loss_v, model, loss_function, x, y):
    cost_function = loss_function()
    plt.rcParams["figure.figsize"] = (10,5)
    plt.grid()
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.title('Convergence of Training')
    plt.plot(range(1,len(loss_t) + 1),loss_t, color='blue', label='Training Loss')
    plt.plot(range(1,len(loss_t) + 1),loss_v, color='red', label='Validation Loss')
    plt.legend()
    plt.ylim([0.0,0.25])
    plt.show()
    print("Final Training Loss = {} | Final Validation Loss = {}".format(loss_t[-1], loss_v[-1]))
    
    x_n = normalize(x, x.max(0,keepdim=True)[0], x.min(0,keepdim=True)[0])
    y_n = normalize(y, y.max(0,keepdim=True)[0], y.min(0,keepdim=True)[0])
    print("Model MSE Loss for whole dataset = {}".format(cost_function(model(x_n) , y_n)))
    

In [14]:
#Normalization Functions
def normalize(x, xmax, xmin):
    return (x - xmin) / (xmax - xmin)

def denormalize(x, xmax, xmin):
    return (x * (xmax - xmin)) + xmin


In [17]:
#Define Constructs
epochs = 200
learn_rate = 1e-3
neural_net = nn.Sequential(OrderedDict([
    ('Layer_1_Model', nn.Linear(5,8)),
    ('Layer_1_Activation', nn.Tanh()),
    ('Layer_2_Model', nn.Linear(5,8)),
    ('Layer_2_Activation', nn.Tanh()),
    ('Layer_3_Model', nn.Linear(5,8)),
    ('Layer_3_Activation', nn.Tanh()),
    ('Output_Model', nn.Linear(8,1))
    ]))
optimizer = optim.Adam(neural_net.parameters(), lr=learn_rate)
loss_function = nn.MSELoss


In [ ]:
%%time
#Perform the Training
loss_t, loss_v = training_loop(
    epochs = epochs,
    optimizer = optimizer,
    model = neural_net,
    loss_function = nn.MSELoss,
    x_t = x_t,
    x_v = x_v,
    y_t = y_t,
    y_v = y_v)